# Training Example: Vanilla vs Cyclical

In [1]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import datasets
from tensorflow.keras import layers, models
import numpy as np
from utils import get_cycle_sizes, get_categ_ind_loss, CyclicalTrain

In [2]:
def get_cifar10_data():
    (train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
    train_images, test_images = (train_images.astype("float32") / 255.0, test_images.astype("float32") / 255.0)
    train_images = train_images.reshape((train_images.shape[0], 32, 32, 3))
    test_images = test_images.reshape((test_images.shape[0], 32, 32, 3))

    train_labels = to_categorical(train_labels)
    test_labels = to_categorical(test_labels)

    return train_images, train_labels, test_images, test_labels


def get_cifar10_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), input_shape=(32, 32, 3)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation("relu"))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation("relu"))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation("relu"))
    model.add(layers.Flatten())
    model.add(layers.Dense(10, activation="softmax"))

    model.compile(
        optimizer='adam',
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )

    model.summary()

    return model

## Get cifar10 dataset and build model
You can try out your own model and dataset by modifying the cell below.

In [3]:
train_images, train_labels, test_images, test_labels = get_cifar10_data()
model = get_cifar10_model()
EPOCHS = 30
BATCH_SIZE = 128

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 30, 30, 32)       128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 13, 13, 64)       2

## Train Vanilla Model with cifar10

In [4]:
vanilla_history = model.fit(train_images, train_labels, 
                            validation_data = (test_images, test_labels),
                            epochs = EPOCHS, batch_size = BATCH_SIZE)

Epoch 1/30
391/391 [==============================] - 7s 9ms/step - loss: 1.3121 - accuracy: 0.5340 - val_loss: 2.2338 - val_accuracy: 0.3157
Epoch 2/30
391/391 [==============================] - 3s 8ms/step - loss: 0.9528 - accuracy: 0.6663 - val_loss: 1.0732 - val_accuracy: 0.6320
Epoch 3/30
391/391 [==============================] - 3s 9ms/step - loss: 0.8196 - accuracy: 0.7146 - val_loss: 1.2582 - val_accuracy: 0.5729
Epoch 4/30
391/391 [==============================] - 3s 9ms/step - loss: 0.7317 - accuracy: 0.7460 - val_loss: 1.0179 - val_accuracy: 0.6553
Epoch 5/30
391/391 [==============================] - 3s 8ms/step - loss: 0.6674 - accuracy: 0.7696 - val_loss: 1.1553 - val_accuracy: 0.6022
Epoch 6/30
391/391 [==============================] - 3s 8ms/step - loss: 0.6159 - accuracy: 0.7851 - val_loss: 2.0346 - val_accuracy: 0.4871
Epoch 7/30
391/391 [==============================] - 3s 8ms/step - loss: 0.5690 - accuracy: 0.8042 - val_loss: 1.0146 - val_accuracy: 0.6691
Epoch 

## Define Cyclical Hyperparameters

In [5]:
start_percent = 0.25
end_percent = 1
multiplier = 0.50

## Get scores for each sample

In [6]:
losses = get_categ_ind_loss(model, train_images, train_labels, batch_size = BATCH_SIZE)
scores = 1 / losses

## Create CyclicalTrain Using Vanilla Scores

In [7]:
cyclical_model, current_max, result_dict = CyclicalTrain(
    model=get_cifar10_model(),
    x=train_images,
    y=train_labels,
    data_sizes=get_cycle_sizes(start_percent, end_percent, multiplier, EPOCHS),
    scores=scores,
    batch_size=BATCH_SIZE,
    callbacks=None,
    verbose=1,
    data=(test_images, test_labels)
)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 30, 30, 32)       128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 13, 13, 64)       2

98/98 [==============================] - 2s 16ms/step - loss: 0.0091 - accuracy: 0.9998 - val_loss: 1.0235 - val_accuracy: 0.7165
Current Max Val Acc 0.7255
196/196 [==============================] - 3s 14ms/step - loss: 0.0452 - accuracy: 0.9907 - val_loss: 1.0142 - val_accuracy: 0.7272
Current Max Val Acc 0.7272
391/391 [==============================] - 5s 11ms/step - loss: 0.3796 - accuracy: 0.8694 - val_loss: 1.1344 - val_accuracy: 0.6750
Current Max Val Acc 0.7272
196/196 [==============================] - 3s 16ms/step - loss: 0.0575 - accuracy: 0.9874 - val_loss: 0.9379 - val_accuracy: 0.7407
Current Max Val Acc 0.7407
98/98 [==============================] - 2s 17ms/step - loss: 0.0084 - accuracy: 0.9991 - val_loss: 0.9223 - val_accuracy: 0.7408
Current Max Val Acc 0.7408
196/196 [==============================] - 3s 13ms/step - loss: 0.0390 - accuracy: 0.9920 - val_loss: 1.2382 - val_accuracy: 0.6901
Current Max Val Acc 0.7408
391/391 [==============================] - 4s 11ms

## Report outcomes

In [10]:
# Log highest accuracy for vanilla and cyclical models
print("Vanilla Highest Accuracy:", round(max(vanilla_history.history['val_accuracy']),4))
print("Cyclical Highest Accuracy:", round(max(result_dict['val_accuracy']),4))

# Log highest 3 accuracies for vanilla and cyclical models
print("Vanilla Highest 3 Accuracies:", sorted(np.round(vanilla_history.history['val_accuracy'],4), reverse = True)[:3])
print("Cyclical Highest 3 Accuracies:", sorted(np.round(result_dict['val_accuracy'],4), reverse = True)[:3])

Vanilla Highest Accuracy: 0.703
Cyclical Highest Accuracy: 0.7408
Vanilla Highest 3 Accuracies: [0.703, 0.7027, 0.6999]
Cyclical Highest 3 Accuracies: [0.7408, 0.7407, 0.7404]
